# Stable-Diffusion-Comfyui Kaggle
**venv 版本：torch: 2.0.1+cu118  •  xformers: 0.0.20**

-----

In [1]:
from pathlib import Path
import subprocess
import pandas as pd
import shutil
import os
import time
import re
import gc
import requests
import zipfile
import threading
import time
import socket
import concurrent.futures
from concurrent.futures import ProcessPoolExecutor

In [2]:
#@title Environment Setup
from pathlib import Path

OPTIONS = {}

UPDATE_COMFY_UI = True  #@param {type:"boolean"}
WORKSPACE = '/kaggle/working/ComfyUI'
OPTIONS['UPDATE_COMFY_UI'] = UPDATE_COMFY_UI

def comfyui():
    time.sleep(0.11)
    %cd /kaggle/working/
    ![ ! -d $WORKSPACE ] && echo -= ready to git clone ComfyUI from QQ2575044704 =- && git clone https://github.com/emjayoh/ComfyUI
    %cd $WORKSPACE
    if OPTIONS['UPDATE_COMFY_UI']:
        !echo -= Updating ComfyUI =-
        !git pull
        !git clone https://github.com/comfyanonymous/ComfyUI
    !git clone https://github.com/ltdrdata/ComfyUI-Manager /kaggle/working/ComfyUI/custom_nodes/ComfyUI-Manager
    !git clone https://github.com/ltdrdata/ComfyUI-Impact-Pack.git /kaggle/working/ComfyUI/custom_nodes/ComfyUI-Impact-Pack
    !git clone https://github.com/SeargeDP/SeargeSDXL.git /kaggle/working/ComfyUI/custom_nodes/SeargeSDXL
    !git clone https://github.com/ssitu/ComfyUI_UltimateSDUpscale --recursive /kaggle/working/ComfyUI/custom_nodes/sd-upscale
    !git clone https://github.com/WASasquatch/was-node-suite-comfyui /kaggle/working/ComfyUI/custom_nodes/was-node-suite-comfyui
    !git clone https://github.com/pythongosssss/ComfyUI-Custom-Scripts /kaggle/working/ComfyUI/custom_nodes/ComfyUI-Custom-Scripts
    !git clone https://github.com/twri/sdxl_prompt_styler /kaggle/working/ComfyUI/custom_nodes/sdxl_prompt_styler
    !git clone https://github.com/failfa-st/failfast-comfyui-extensions /kaggle/working/ComfyUI/custom_nodes/failfast
    !git clone https://github.com/civitai/comfy-nodes /kaggle/working/ComfyUI/custom_nodes/comfy-nodes
    !git clone https://github.com/asagi4/comfyui-prompt-control /kaggle/working/ComfyUI/custom_nodes/prompt-control
    !git clone https://github.com/adieyal/comfyui-dynamicprompts /kaggle/working/ComfyUI/custom_nodes/dynamic-prompt
    !git clone https://github.com/mihaiiancu/ComfyUI_Inpaint /kaggle/working/ComfyUI/custom_nodes/inpaint
    !git clone https://github.com/AIrjen/OneButtonPrompt /kaggle/working/ComfyUI/custom_nodes/one-button-prompt
    !git clone https://github.com/coreyryanhanson/ComfyQR /kaggle/working/ComfyUI/custom_nodes/qr
    !git clone https://github.com/cubiq/ComfyUI_essentials /kaggle/working/ComfyUI/custom_nodes/cui-essentials
    !git clone https://github.com/phineas-pta/comfyui-auto-nodes-layout /kaggle/working/ComfyUI/custom_nodes/auto-nodes-layout
    !git clone https://github.com/jmkl/ComfyUI-ricing /kaggle/working/ComfyUI/custom_nodes/cui-ricing
    !git clone https://github.com/Ttl/ComfyUi_NNLatentUpscale /kaggle/working/ComfyUI/custom_nodes/nn-latent
    !git clone https://github.com/Lerc/canvas_tab /kaggle/working/ComfyUI/custom_nodes/canvas-tab
    !git clone https://github.com/dagthomas/comfyui_dagthomas /kaggle/working/ComfyUI/custom_nodes/dag-prompt
        # T2I-Adapter
#     !wget -c https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_depth_sd14v1.pth -P /kaggle/working/ComfyUI/models/controlnet
#     !wget -c https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_seg_sd14v1.pth -P /kaggle/working/ComfyUI/models/controlnet
#     !wget -c https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_sketch_sd14v1.pth -P /kaggle/working/ComfyUI/models/controlnet
#     !wget -c https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_keypose_sd14v1.pth -P /kaggle/working/ComfyUI/models/controlnet
#     !wget -c https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_openpose_sd14v1.pth -P /kaggle/working/ComfyUI/models/controlnet
#     !wget -c https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_color_sd14v1.pth -P /kaggle/working/ComfyUI/models/controlnet
#     !wget -c https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_canny_sd14v1.pth -P /kaggle/working/ComfyUI/models/controlnet

# T2I Styles Model
#     !wget -c https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_style_sd14v1.pth -P /kaggle/working/ComfyUI/models/style_models/

    # CLIPVision model (needed for styles model)
#     !wget -c https://huggingface.co/openai/clip-vit-large-patch14/resolve/main/pytorch_model.bin -O /kaggle/working/ComfyUI/models/clip_vision/clip_vit14.bin


    #  ControlNet
#     !wget -c https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11e_sd15_ip2p_fp16.safetensors -P /kaggle/working/ComfyUI/models/controlnet/
#     !wget -c https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11e_sd15_shuffle_fp16.safetensors -P /kaggle/working/ComfyUI/models/controlnet/
#     !wget -c https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_canny_fp16.safetensors -P /kaggle/working/ComfyUI/models/controlnet/
#     !wget -c https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11f1p_sd15_depth_fp16.safetensors -P /kaggle/working/ComfyUI/models/controlnet/
#     !wget -c https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_inpaint_fp16.safetensors -P /kaggle/working/ComfyUI/models/controlnet/
#     !wget -c https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_lineart_fp16.safetensors -P /kaggle/working/ComfyUI/models/controlnet/
#     !wget -c https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_mlsd_fp16.safetensors -P /kaggle/working/ComfyUI/models/controlnet/
#     !wget -c https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_normalbae_fp16.safetensors -P /kaggle/working/ComfyUI/models/controlnet/
#     !wget -c https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_openpose_fp16.safetensors -P /kaggle/working/ComfyUI/models/controlnet/
#     !wget -c https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_scribble_fp16.safetensors -P /kaggle/working/ComfyUI/models/controlnet/
#     !wget -c https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_seg_fp16.safetensors -P /kaggle/working/ComfyUI/models/controlnet/
#     !wget -c https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_softedge_fp16.safetensors -P /kaggle/working/ComfyUI/models/controlnet/
#     !wget -c https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15s2_lineart_anime_fp16.safetensors -P /kaggle/working/ComfyUI/models/controlnet/
#     !wget -c https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11u_sd15_tile_fp16.safetensors -P /kaggle/working/ComfyUI/models/controlnet/


    # Controlnet Preprocessor nodes by Fannovel16
#     !cd custom_nodes && git clone https://github.com/Fannovel16/comfy_controlnet_preprocessors; cd comfy_controlnet_preprocessors && python install.py
#     %cd $WORKSPACE
    # GIT
#     !git clone https://github.com/Fannovel16/comfy_controlnet_preprocessors /kaggle/working/ComfyUI/custom_nodes/comfy_controlnet_preprocessors
    # GLIGEN
#     !wget -c https://huggingface.co/comfyanonymous/GLIGEN_pruned_safetensors/resolve/main/gligen_sd14_textbox_pruned_fp16.safetensors -P /kaggle/working/ComfyUI/models/gligen/
    

    # ESRGAN upscale model
    #!wget -c https://github.com/xinntao/Real-ESRGAN/releases/download/v0.1.0/RealESRGAN_x4plus.pth -P /kaggle/working/ComfyUI/models/upscale_models/
    #!wget -c https://huggingface.co/sberbank-ai/Real-ESRGAN/resolve/main/RealESRGAN_x2.pth -P /kaggle/working/ComfyUI/models/upscale_models/
    #!wget -c https://huggingface.co/sberbank-ai/Real-ESRGAN/resolve/main/RealESRGAN_x4.pth -P /kaggle/working/ComfyUI/models/upscale_models/



def prepare():
    time.sleep(0.5)
    !npm install -g localtunnel & npm i -g pm2
#   !apt-get update
    !apt -y install aria2
    !pip install gdown
    import gdown
#     print('正在下载sdxl 1.0基本模型,大小6.9G')
#     !mkdir -p /kaggle/working/ComfyUI/models/checkpoints/
#     !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M "https://huggingface.co/stabilityai/stable-diffusion-xl-base-1.0/resolve/main/sd_xl_base_1.0.safetensors"  -d /kaggle/working/ComfyUI/models/checkpoints/ -o sd_xl_base_1.0.safetensors
#     print('正在下载sdxl 1.0优化模型,大小6.5G')
#     print('DOWNLOADING MODELS...')
#     !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M "https://huggingface.co/stabilityai/stable-diffusion-xl-refiner-1.0/resolve/main/sd_xl_refiner_1.0.safetensors"  -d /kaggle/working/ComfyUI/models/checkpoints/ -o sd_xl_refiner_1.0.safetensors
#     gdown "https://drive.google.com/file/d/11KbAr05Y-FGLuc2n9PC24Qav84dR6ttg/view?usp=drive_link" -O /kaggle/working/ComfyUI/models/checkpoints/aimee.ckpt
#     gdown "https://drive.google.com/file/d/1GJJDY_2m542PhVjmZ0NoF4vGFxaHb450/view?usp=sharing" -O /kaggle/working/ComfyUI/models/checkpoints/aimee.yaml
#     gdown.download("https://drive.google.com/file/d/11KbAr05Y-FGLuc2n9PC24Qav84dR6ttg/view?usp=drive_link", "/kaggle/working/ComfyUI/models/checkpoints/aimee.ckpt", quiet=False)
#     gdown.download("https://drive.google.com/file/d/1GJJDY_2m542PhVjmZ0NoF4vGFxaHb450/view?usp=sharing", "/kaggle/working/ComfyUI/models/checkpoints/aimee.yaml", quiet=False)
#     !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://civitai.com/api/download/models/131579  -d /kaggle/working/ComfyUI/models/checkpoints -o rundiffusion-xl.safetensors
#     !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M  https://huggingface.co/kamaltdin/UltraSharp/resolve/main/4x-UltraSharp.pth -d /kaggle/working/ComfyUI/models/upscale_models -o 4x-UltraSharp.pth
    
    
def venv_install():
    %cd /opt/conda/envs
    if os.path.exists('venv'):
        print('venv exists')
    else:
        %cd /kaggle
        if not os.path.exists('venv.tar.gz'):
            !wget https://huggingface.co/datasets/sukaka/venv_ai_drow/resolve/main/sd_webui/sd_webui_torch201_cu118_xf20.tar.gz -O venv.tar.gz
        %cd /opt/conda/envs/
        !mkdir venv
        %cd venv
        !apt -y install -qq pigz
        !pigz -dc -p 500 /kaggle/venv.tar.gz | tar xf -
        !source /opt/conda/bin/activate venv



In [3]:
def libtcmalloc():
    if os.path.exists('/kaggle/temp'):
        os.chdir('/kaggle')
        os.chdir('temp')
        os.environ["LD_PRELOAD"] = "libtcmalloc.so"
        print('内存优化已安装')
    else:
        
        #if use_frpc:
            #!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/datasets/ACCA225/Frp/resolve/main/frpc -d /kaggle/working/frpc -o frpc
        os.system('pip install -q pyngrok ')
        os.chdir('/kaggle')
        os.makedirs('temp', exist_ok=True)
        os.chdir('temp')
        os.system('wget -qq  http://launchpadlibrarian.net/367274644/libgoogle-perftools-dev_2.5-2.2ubuntu3_amd64.deb')
        os.system('wget -qq  https://launchpad.net/ubuntu/+source/google-perftools/2.5-2.2ubuntu3/+build/14795286/+files/google-perftools_2.5-2.2ubuntu3_all.deb')
        os.system('wget -qq  https://launchpad.net/ubuntu/+source/google-perftools/2.5-2.2ubuntu3/+build/14795286/+files/libtcmalloc-minimal4_2.5-2.2ubuntu3_amd64.deb')
        os.system('wget -qq  https://launchpad.net/ubuntu/+source/google-perftools/2.5-2.2ubuntu3/+build/14795286/+files/libgoogle-perftools4_2.5-2.2ubuntu3_amd64.deb')
        os.system('apt install -qq libunwind8-dev -y')
        !dpkg -i *.deb
        os.environ["LD_PRELOAD"] = "libtcmalloc.so"
        !rm *.deb
        print('内存优化已安装')

In [4]:
ngrok = True
def ngrok():
    !pip install pyngrok
    %cd /kaggle/working/hello-ngrok
    from pyngrok import conf, ngrok
    ngrokToken = '2UBdQ9zkOd0EuztpC4nVbGr6PFG_6oTb5Egy5pV4B3NXDPfHh'
    conf.get_default().auth_token = ngrokToken
    conf.get_default().monitor_thread = False
    ssh_tunnels = ngrok.get_tunnels(conf.get_default())
    if len(ssh_tunnels) == 0:
        ssh_tunnel = ngrok.connect(8188)
        print('address：'+ssh_tunnel.public_url)
    else:
        print('address：'+ssh_tunnels[0].public_url)
    


import subprocess
import threading
import time
import socket
import urllib.request


def iframe_thread(port):
    while True:
        time.sleep(0.5)
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        result = sock.connect_ex(('127.0.0.1', port))
        if result == 0:
            break
        sock.close()
        !sleep 8
        print("\nComfyUI finished loading, trying to launch localtunnel (if it gets stuck here localtunnel is having issues)\n")
        print("\033[91m请通过公网IP进入ComfyUI，你的公网IP地址是:", urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"), "\033[0m")
        p = subprocess.Popen(["lt", "--port", "{}".format(port)], stdout=subprocess.PIPE)
        for line in p.stdout:
            print(line.decode(), end='')

def start():
    threading.Thread(target=iframe_thread, daemon=True, args=(8188,)).start()
    %cd
    %cd /kaggle/working/ComfyUI
    !pm2 log & /opt/conda/envs/venv/bin/python3 main.py --cuda-malloc

# 执行区域，输出结果在下方看

In [5]:
'''
执行函数
'''
!echo '采用多线程启动，最大化利用CPU，加快启动速度'

with concurrent.futures.ThreadPoolExecutor() as executor:
    future1 = executor.submit(comfyui) # ComfyUI
    future2 = executor.submit(venv_install) # VENV
    future4 = executor.submit(prepare) # Linux

##################
## UPSCALE MODELZ
##################
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M  https://github.com/Phhofm/models/raw/main/4xLSDIR/4xLSDIR.pth -d /kaggle/working/ComfyUI/models/upscale_models -o 4xLSDIR.pth
!wget -c https://huggingface.co/sberbank-ai/Real-ESRGAN/resolve/main/RealESRGAN_x2.pth -P /kaggle/working/ComfyUI/models/upscale_models
!wget -c https://huggingface.co/sberbank-ai/Real-ESRGAN/resolve/main/RealESRGAN_x4.pth -P /kaggle/working/ComfyUI/models/upscale_models
!wget -c https://huggingface.co/uwg/upscaler/resolve/main/ESRGAN/4x-UltraSharp.pth -P /kaggle/working/ComfyUI/models/upscale_models
!wget -c https://huggingface.co/uwg/upscaler/resolve/main/ESRGAN/4x_RealisticRescaler_100000_G.pth -P /kaggle/working/ComfyUI/models/upscale_models
!wget -c https://huggingface.co/utnah/LDSR/resolve/main/4x_Nickelback_70000G.pth -P /kaggle/working/ComfyUI/models/upscale_models
!wget -c https://github.com/JingyunLiang/SwinIR/releases/download/v0.0/001_classicalSR_DF2K_s64w8_SwinIR-M_x3.pth -P /kaggle/working/ComfyUI/models/upscale_models
!wget -c https://github.com/JingyunLiang/SwinIR/releases/download/v0.0/001_classicalSR_DF2K_s64w8_SwinIR-M_x2.pth -P /kaggle/working/ComfyUI/models/upscale_models
!wget -c https://github.com/xinntao/Real-ESRGAN/releases/download/v0.2.1/RealESRGAN_x2plus.pth -P /kaggle/working/ComfyUI/models/upscale_models


###############
## CHECKPOINTS
###############
## ZAVYCHROMA
# !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://civitai.com/api/download/models/158371 -d /kaggle/working/ComfyUI/models/checkpoints -o zavy-xl.safetensors

## [Lah] Mysterious
# !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://civitai.com/api/download/models/149020 -d /kaggle/working/ComfyUI/models/checkpoints -o myst-xl.safetensors

## RUNDIFFUSION
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://civitai.com/api/download/models/131579  -d /kaggle/working/ComfyUI/models/checkpoints -o rundiffusion-xl.safetensors

## COPAX v6
# !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://civitai.com/api/download/models/164378 -d /kaggle/working/ComfyUI/models/checkpoints -o COPAXv6.safetensors

## JUGGERNAUT XL
# !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://civitai.com/api/download/models/159123 -d /kaggle/working/ComfyUI/models/checkpoints -o hephaistos.safetensors

## HEPHAISTOS XL
# !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://civitai.com/api/download/models/162509 -d /kaggle/working/ComfyUI/models/checkpoints -o jugg.safetensors
# https://civitai.com/api/download/models/164826?type=VAE

## RUNDIFFUSION
# !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://civitai.com/api/download/models/131579 -d /kaggle/working/ComfyUI/models/checkpoints -o rundiffusion.safetensors
## YAMERS MIX + SDXL VAE
# !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://civitai.com/api/download/models/170252 -d /kaggle/working/ComfyUI/models/checkpoints -o yamers-real.safetensors
# !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://civitai.com/api/download/models/164826?type=VAE -d /kaggle/working/ComfyUI/models/vae -o sdxl-vae.safetensors

## REALIST XL
# !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://civitai.com/api/download/models/134969  -d /dels/checkpoints -o realist-xl.safetensors

##############
## LORAS 
##############

## GENERAL MODS (DETAIL AND PHOTOREALISM/ART)
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://civitai.com/api/download/models/141133 -d /kaggle/working/ComfyUI/models/loras -o real.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://civitai.com/api/download/models/152309 -d /kaggle/working/ComfyUI/models/loras -o artful.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://civitai.com/api/download/models/135867 -d /kaggle/working/ComfyUI/models/loras -o detail.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://civitai.com/api/download/models/169002 -d /kaggle/working/ComfyUI/models/loras -o fractal.safetensors

## TRIGGER: "LogoRedAF"
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://civitai.com/api/download/models/136078 -d /kaggle/working/ComfyUI/models/loras -o logos.safetensors
    
## TRIGGER: "Explosion Artstyle"
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://civitai.com/api/download/models/161516 -d /kaggle/working/ComfyUI/models/loras -o explosion.safetensors

## TRIGGER: "building", "architecture"
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://civitai.com/api/download/models/147752 -d /kaggle/working/ComfyUI/models/loras -o arch.safetensors

## TRIGGER: "Tarot Card Style"
## https://civitai.com/api/download/models/170833
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://civitai.com/api/download/models/170833 -d /kaggle/working/ComfyUI/models/loras -o tarot.safetensors


## BUGZ
# !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ogstradamus/bugz/resolve/main/bugz.ckpt -d /kaggle/working/ComfyUI/models/checkpoints -o bugz.ckpt
# !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://civitai.com/api/download/models/46259?type=Config -d /kaggle/working/ComfyUI/models/checkpoints -o bugz.yaml

## AIMEE
# !gdown https://drive.google.com/uc?id=11KbAr05Y-FGLuc2n9PC24Qav84dR6ttg -O /kaggle/working/ComfyUI/models/checkpoints/aimee.ckpt
# !gdown https://drive.google.com/uc?id=1GJJDY_2m542PhVjmZ0NoF4vGFxaHb450 -O /kaggle/working/ComfyUI/models/checkpoints/aimee.yaml

## JOSH
# https://drive.google.com/file/d/101kSa4b2j3iaH4wes7k6178cZHb3vYQv/view?usp=drive_link
# !gdown https://drive.google.com/uc?id=101kSa4b2j3iaH4wes7k6178cZHb3vYQv -O /kaggle/working/ComfyUI/models/checkpoints/josh.ckpt
# !gdown https://drive.google.com/uc?id=1GJJDY_2m542PhVjmZ0NoF4vGFxaHb450  -O /kaggle/working/ComfyUI/models/checkpoints/josh.yaml
    
## MATT
# https://drive.google.com/file/d/158mqQIKkS084RMxhyCzy6uMMcNpFJYQe/view?usp=drive_link
# !gdown https://drive.google.com/uc?id=158mqQIKkS084RMxhyCzy6uMMcNpFJYQe -O /kaggle/working/ComfyUI/models/checkpoints/matt.ckpt
# !gdown https://drive.google.com/uc?id=1GJJDY_2m542PhVjmZ0NoF4vGFxaHb450  -O /kaggle/working/ComfyUI/models/checkpoints/matt.yaml

## TERA
# https://drive.google.com/file/d/12RxKyy988BY-s5I5C-BESLl-dBPoEtyR/view?usp=drive_link
# !gdown https://drive.google.com/uc?id=12RxKyy988BY-s5I5C-BESLl-dBPoEtyR -O /kaggle/working/ComfyUI/models/checkpoints/tera.ckpt
# !gdown https://drive.google.com/uc?id=1GJJDY_2m542PhVjmZ0NoF4vGFxaHb450  -O /kaggle/working/ComfyUI/models/checkpoints/tera.yaml
    
## TIM
# https://drive.google.com/file/d/12jJ2oGb7Ci6ExCMlwARGULySYL5go2kW/view?usp=drive_link
# !gdown https://drive.google.com/uc?id=12jJ2oGb7Ci6ExCMlwARGULySYL5go2kW -O /kaggle/working/ComfyUI/models/checkpoints/tim.ckpt
# !gdown https://drive.google.com/uc?id=1GJJDY_2m542PhVjmZ0NoF4vGFxaHb450  -O /kaggle/working/ComfyUI/models/checkpoints/tim.yaml
    
## JAMES
# https://drive.google.com/file/d/1VnA9ziabW02E-GdN8svbmFGmnBSD22QO/view?usp=drive_link
# !gdown https://drive.google.com/uc?id=1VnA9ziabW02E-GdN8svbmFGmnBSD22QO-O /kaggle/working/ComfyUI/models/checkpoints/james.ckpt

采用多线程启动，最大化利用CPU，加快启动速度
/opt/conda/envs
/kaggle
/kaggle/working
--2023-09-26 21:05:19--  https://huggingface.co/datasets/sukaka/venv_ai_drow/resolve/main/sd_webui/sd_webui_torch201_cu118_xf20.tar.gz
Resolving huggingface.co (huggingface.co)... 18.172.134.124, 18.172.134.4, 18.172.134.24, ...
Connecting to huggingface.co (huggingface.co)|18.172.134.124|:443... connected.
-= ready to git clone ComfyUI from QQ2575044704 =-
Cloning into 'ComfyUI'...
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/93/61/936160f9623602ad97a9fe4c639531b59f4fe39854fcc22d75692344fb5dfbe2/609336a63928c38f99b0f842dd5d08e1bf255a8add61cbe82301ca5611831b34?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27sd_webui_torch201_cu118_xf20.tar.gz%3B+filename%3D%22sd_webui_torch201_cu118_xf20.tar.gz%22%3B&response-content-type=application%2Fgzip&Expires=1696021519&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTY5NjAyM

In [ ]:
libtcmalloc()
start()

The following additional packages will be installed:
  liblzma-dev libunwind8
Suggested packages:
  liblzma-doc
The following NEW packages will be installed:
  liblzma-dev libunwind-dev
The following packages will be upgraded:
  libunwind8


dpkg-preconfigure: unable to re-open stdin: No such file or directory


1 upgraded, 2 newly installed, 0 to remove and 73 not upgraded.
Need to get 2096 kB of archives.
After this operation, 6755 kB of additional disk space will be used.
(Reading database ... 113908 files and directories currently installed.)
Preparing to unpack .../libunwind8_1.3.2-2build2.1_amd64.deb ...
Unpacking libunwind8:amd64 (1.3.2-2build2.1) over (1.3.2-2build2) ...
Selecting previously unselected package liblzma-dev:amd64.
Preparing to unpack .../liblzma-dev_5.2.5-2ubuntu1_amd64.deb ...
Unpacking liblzma-dev:amd64 (5.2.5-2ubuntu1) ...
Selecting previously unselected package libunwind-dev:amd64.
Preparing to unpack .../libunwind-dev_1.3.2-2build2.1_amd64.deb ...
Unpacking libunwind-dev:amd64 (1.3.2-2build2.1) ...
Setting up libunwind8:amd64 (1.3.2-2build2.1) ...
Setting up liblzma-dev:amd64 (5.2.5-2ubuntu1) ...
Setting up libunwind-dev:amd64 (1.3.2-2build2.1) ...
Processing triggers for man-db (2.10.2-1) ...
Processing triggers for libc-bin (2.35-0ubuntu3.1) ...
Selecting previous

In [ ]:
!zip -r file.zip /kaggle/working/ComfyUI/output